# Dependency Tree und Noun Chunks

Einzelne Wörter der Sätze hängen voneinander ab. Daher gibt es einen sog. *Abhängigkeitsbaum*. Diesen werden wir nun bestimmen.

Mithilfe sog. *Noun Chunks* lassen sich grammatikalisch zusammengehörige Wörter herausfinden. Diese schauen wir uns anschließend an.

## Datenbasis

Wie immer startest du wir mit den beiden Absätzen aus der Neujahrsmeldung:

In [ ]:
p1 = "Doch das Ende des Jahres 2020 birgt auch Hoffnung, dass durch die Vakzinen \
gegen Covid-19 wieder Normalität einkehre – wie immer die auch aussehen mag \
– und wir uns um anderes Dringliches kümmern oder einfach entspannen \
können. Und dass durch den im Januar anstehenden Bewohnerwechsel im \
Weißen Haus zu Washington D.C. das offizielle Herumgetrumpel auf dem \
gesunden Menschenverstand ein Ende finden möge."

p2 = "Wir, das gesamte Team von heise online und die Redaktionen von c't, iX, \
Technology Review, Mac & i, c't Digitale Fotografie, Make:, Techstage und \
Telepolis sowie heise Security, heise Developer und heise Autos wünschen Ihnen \
ein friedliches und freudvolles Jahr 2021. Wir wünschen Ihnen, dass Sie nicht \
vergeblich hoffen und dass Ihre Vorsätze erfüllt werden, auf dass Sie gesund \
bleiben oder genesen."

Und lässt die durch `spacy` analysieren:

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg")
d1 = nlp(p1)
d2 = nlp(p2)

## Satzanalyse und Interpretation der Abhängigkeiten

In der linguistischen Analyse hat `spacy` auch die Satzglieder und deren Abhängigkeit bestimmt. *Satzglied* ist hier mit etwas Vorsicht zu genießen, da `spacy` das für jeden Token bestimmt, welche Funktion er im Satz erfüllt. Ein Satzglied (wie im Deutschunterricht) kann allerdings durchaus aus mehreren Wörtern (Tokens) bestehen.

Als erstes schreibst du eine Funktion, die für jeden Token seine Stellung im Satz und die Abhängigkeit zu einem anderen Satzglied ermittelt:

In [ ]:
import pandas as pd

pd.set_option("max.rows", None)
pd.set_option('display.max_colwidth', None)

def dep(d):
    res = []
    for token in d:
        res.append((token.text, token.dep_, token.head.dep_))

    return pd.DataFrame(res, columns=["Text", "Dep", "Head Dep"])

Da die Sätze sehr lang sind, schaust du dir zuerst den zweiten Satz des ersten Absatzes an:

In [ ]:
dep11 = dep(list(d1.sents)[1])
dep11

Das Ergebnis sieht etwas unübersichtlich aus. Am einfachsten beginnst du bei dem Wert `ROOT` im `Dep`-Feld. Das ist die *Wurzel* des Satzes. Dann kannst du nachsehen, welche anderen Tokens direkt mit der Wurzel zusammenhängen, diese haben dann `ROOT` im Feld `Head Dep`:

In [ ]:
dep11[(dep11["Dep"] == "ROOT") | (dep11["Head Dep"] == "ROOT")]

So kannst du dich weiter durch den Satz bewegen.

`spacy.explain` kann dir auch bei der Interpretation der *Dep*-Felder helfen:

In [ ]:
res = []
for d in dep11["Dep"].unique():
    res.append({"Dep": d, "Explanation": spacy.explain(d)})
pd.DataFrame(res)

## Visualisierung

Mithilfe von `displacy` kannst du den Abhängkeitsbaum auch direkt visualisieren. Bei längeren Dokumenten funktioniert das leider nur eingeschränkt gut:

In [ ]:
from spacy import displacy

displacy.render(d1, style="dep", jupyter=True)

Besser ist es, du visualisierst die einzelnen Sätz separat. Damit ist auch das Scrolling in den Sätzen getrennt voneinander möglich. Mit der Option `options={"compact": True}` erreichst du eine etwas kompaktere Darstellung ohne Kreisbögen:

In [ ]:
for s in d1.sents:
    displacy.render(s, style="dep", jupyter=True, options={"compact": True})

## Noun chunks

Noun chunks sind eine Art *Satzbausteine* und bestehen nur aus Substantiven oder Pronomen kombiniert mit Artikeln und Adjektiven. Technisch gesprochen haben sie ein Substantiv als *Kopf*. Du kannst dir *Noun chunks* als Substantive mit den dazugehörigen *Beschreibungswörtern* vorstellen.

Dafür kannst du in einer Funktion über alle *Noun Chunks* iterieren, deren Text anzeigen, die Wurzel und den Kopf der Wurzel.

In [ ]:
def noun_chunks(d):
    res = []
    for token in d.noun_chunks:
        res.append((token.text, token.root.text, token.root.dep_, token.root.head.text))

    return pd.DataFrame(res, columns=["Text", "Root Text", "Root Dep", "Root Head Text"])

In [ ]:
noun_chunks(d1)

Unter den *Noun Chunks* kannst du dir die Bausteine eines Satzes oder die *Bedeutungseinheiten* in Form von Substantiven vorstellen.